In [117]:
import src.constants; reload(src.constants)
from src.constants import fn_frsq_venues_dc, fn_segments_dc, epsg_dc, fn_osm_db_dc
from src.constants import fn_mapping_for_fs, fn_mapping_for_osm

import src.geom_helper; reload(src.geom_helper)
from src.geom_helper import bfr_20m, objs_near_segs, objs_near_segs_store

import geopandas as gp
import pandas as pd

import src.poi; reload(src.poi)
from src.poi import *

# TODO
0. [] update frsq poi mapping
1. [x] mapped frsq venues to poi categories --> filter frsq venues by obj_near_segment
2. [x] mapped osm data to poi categories
4. [] filter osm data by obj_near_segment
3. [] remove overlap between osm and frsq
4. [] Visalization:
    - [] poi categories distribution
        1. [x] frsq venues near segments
        2. [x] osm venues near segments
        3. [] final poi near segments(after removing overlap)
    - [] poi categories spatial distribution

In [103]:
data_dir = 'data/'
path_frsq_venues_dc = data_dir + fn_frsq_venues_dc
path_osm_db_dc = data_dir + fn_osm_db_dc
path_segments_dc = data_dir + fn_segments_dc
path_mapping_for_fs = data_dir+fn_mapping_for_fs
path_mapping_for_osm = data_dir+fn_mapping_for_osm


In [118]:
frsq_venues_dc = gp.read_file(path_frsq_venues_dc)
segments_dc = gp.read_file(path_segments_dc)
poi_frsq_dc = map_frsq_venues_to_poi_category(frsq_venues_dc, path_mapping_for_fs, debug=True)
poi_frsq_near_seg_dc = objs_near_segs(poi_frsq_dc, segments_dc, bfr_20m, epsg_dc, output='objs')
print '# venues in city=',poi_frsq_dc.shape[0]
print '# venues near segments=', poi_frsq_near_seg_dc.shape[0]
objs_near_segs_store(poi_frsq_near_seg_dc, path_frsq_venues_dc, path_segments_dc)


===========mapping  frsq venues to poi categories===========
venues without poi category: #venues=3139, #frsq_categories=63
top ten frsq_categories {u'': 1447, u'Building': 882, u'TV Station': 11, u'Exhibit': 51, u'Moving Target': 121, u'Zoo': 14, u'Festival': 6, u'Conference': 7, u'Intersection': 55, u'Road': 450}
# venues in city= 30187
# venues near segments= 18177
wrote obj near segments: data/frsq_venues_dc_near_segments_dc_opendc.geojson


In [5]:
def mapping_for_osm(path_mapping_for_osm):
    result = []
    with open(path_mapping_for_osm) as f:
        category, key, value = '','',''
        for line in f:
            line_ = line
            line = line.split('\t')
            type_ = len(line)
            txt = line[-1].strip()
            if type_==1:
                category = txt
            elif type_==2:
                key = txt
            elif type_==3:
                value = txt
                result.append([category, key, value])
    new_taxonomy = pd.DataFrame(result,columns=['category', 'key', 'value'])
    mapping = pd.Series(new_taxonomy.category.values, 
              index=new_taxonomy.apply(lambda x: '{}={}'.format(x.key, x.value), axis=1).values
             ).to_dict()
    return mapping


In [99]:
def map_osm_to_poi_category(path_osm_db, path_mapping_for_osm, debug=True):
    print '===========mapping osm to poi category==========='
    from src.osm.osmdb_filter import filter_tbtag
    from src.osm.osmdb_constants import FIELDS_TB_TAG
    from src.constants import var_exclude_category_for_osm
    assert FIELDS_TB_TAG==['ot', 'oid', 'key', 'value'], 'Assume fields of table tag are ot,oid,key,value'

    rows = filter_tbtag(path_osm_db)

    mapping = mapping_for_osm(path_mapping_for_osm)
    mapped = []
    for ot, oid, key, value in rows:
        key_value = '{}={}'.format(key.encode('utf-8'), value.encode('utf-8'))
        key_ = key + '=*'
        category = ''
        if key_value in mapping:
            category = mapping[key_value]
        elif key_ in mapping:
            category = mapping[key_]
        if category:
            mapped.append((ot, oid, category))
    
    poi_osm = pd.DataFrame(mapped, columns=FIELDS_TB_TAG[:2]+['category'])
    poi_osm = poi_osm[poi_osm.category!=var_exclude_category_for_osm].groupby(['ot','oid']).agg(set).reset_index()
    
    if debug:
        print '# rows in table tag  =', len(rows)        
        print '# rows after mapping =', len(mapped)
        print '# objs have category =', len(poi_osm)
        print '# objs have category by ot:', poi_osm.ot.value_counts().tolist()
        print '# categories: # objs =', poi_osm.category.apply(len).value_counts().to_dict()
    return poi_osm

poi_osm_dc = map_osm_to_poi_category(path_osm_db_dc, path_mapping_for_osm, debug=True)


===========mapping osm to poi category===========
# rows in table tag  = 1484405
# rows after mapping = 44060
# objs have category = 32808
# objs have category by ot: [20010, 11719, 1079]
# categories: # objs = {1: 32183, 2: 620, 3: 5}


In [100]:
import src.osm.osmdb_filter; reload(src.osm.osmdb_filter)
from src.osm.osmdb_filter import filter_geom_by_otoids_to_gpdf
shapes = filter_geom_by_otoids_to_gpdf(path_osm_db_dc, poi_osm_dc[['ot','oid']].values.tolist())

In [101]:
poi_osm_dc = shapes.merge(poi_osm_dc)

In [102]:
poi_osm_dc

,ot,oid,geometry,category
0,N,49715946,POINT (-77.0176445 38.953198),set([cycling facilities])
1,N,49715951,POINT (-77.01971740000001 38.9531399),set([cycling facilities])
2,N,49715974,POINT (-77.0254044 38.9529676),set([cycling facilities])
3,N,49716326,POINT (-77.0035556 38.8604741),set([cycling facilities])
4,N,49716545,POINT (-77.00767500000001 38.829199),set([cycling facilities])
5,N,49716602,POINT (-77.006002 38.824091),set([cycling facilities])
6,N,49716610,POINT (-77.00563099999999 38.823747),set([cycling facilities])
7,N,49716662,POINT (-77.0090656 38.8875904),set([cycling facilities])
8,N,49716664,POINT (-77.00906240000001 38.8875047),set([cycling facilities])
9,N,49716680,POINT (-77.0090491 38.8838384),set([cycling facilities])


In [121]:
a = objs_near_segs(poi_osm_dc, segments_dc, bfr_20m, epsg_dc, output='objs')

In [123]:
a.ot.value_counts()

W    14389
N     7308
R      878
dtype: int64

In [126]:
rs = poi_osm_dc[poi_osm_dc.ot=='R']
r_pt = []
from shapely.geometry import Point
for _, r in rs.iterrows():
    geom = r.geometry
    for x in geom:
        if isinstance(x, Point):
            r_pt.append((r.name, r.ot, r.oid, x, r.category))
r_pt = gp.GeoDataFrame(r_pt, columns=['index','ot','oid','geometry','category'])

In [128]:
ns = poi_osm_dc[poi_osm_dc.ot=='N']
gp.tools.sjoin(ns, rs)

TopologyException: side location conflict at -76.991639679998158 38.916796608301951


AttributeError: 'PreparedGeometry' object has no attribute 'is_valid'

In [125]:
ns = poi_osm_dc[poi_osm_dc.ot=='N']
gp.tools.sjoin(ns, r_pt)

,ot_left,oid_left,geometry,category_left,index_right,index,ot_right,oid_right,category_right
6979,N,2027049957,POINT (-77.1005091 38.9189329),set([cycling facilities]),6,12710,R,3322742,set([cycling facilities])
7029,N,2129339202,POINT (-77.01745270000001 38.8745415),set([transportation]),4,12674,R,2723846,set([transportation])
7030,N,2129346024,POINT (-77.0184975 38.8765762),set([transportation]),5,12674,R,2723846,set([transportation])


In [97]:
poi_osm_dc.iloc[2989].category |= set(['2'])

In [98]:
poi_osm_dc.iloc[2989]

ot                                              N
oid                                     448821793
geometry    POINT (-77.09086120000001 38.9352604)
category              set([1, 2, transportation])
Name: 2989, dtype: object

In [84]:
set([1,2])|set([12])

{1, 2, 12}

In [10]:
def poi_distribution(poi_frsq, poi_osm, poi=None):
    import numpy as np
    poi_frsq_distr = poi_frsq.category.value_counts().reset_index()
    poi_frsq_distr.columns = ['category','fs']
    categories, counts = np.unique(np.hstack(poi_osm.category.apply(list).apply(np.array).values), return_counts=True)
    poi_osm_distr = pd.DataFrame(zip(categories,counts),columns=['category','osm'])
    print poi_frsq_distr
    print poi_osm_distr
    print poi_frsq_distr.merge(poi_osm_distr)
    
poi_distribution(poi_frsq_dc, poi_osm_dc)

                   category    fs
0      professional service  6620
1   outdoors and recreation  4357
2                      food  3937
3               retail shop  3753
4               no category  3139
5                 residence  1980
6            transportation  1927
7        schools&university  1709
8            nightlife spot  1593
9                       art  1007
10       cycling facilities   165
                  category    osm
0                      art    130
1       cycling facilities  12646
2                     food   1623
3           nightlife spot    278
4  outdoors and recreation   6037
5     professional service   1449
6                residence    583
7              retail shop   1773
8       schools&university    948
9           transportation   7971
                  category    fs    osm
0     professional service  6620   1449
1  outdoors and recreation  4357   6037
2                     food  3937   1623
3              retail shop  3753   1773
4                r